In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Additional libraries
import matplotlib.pyplot as plt # for plots
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold # for splits

from sklearn import metrics # for post-analysis

1. Reading the input data files 

In [ ]:
# reading in data files
data_train = pd.read_csv('../input/mnist_train.csv')
data_test = pd.read_csv('../input/mnist_test.csv')

2. Basic Diagnostics

In [ ]:
print("The shape of TRAINING Data is: ", data_train.shape)
print("The shape of TEST Data is: ", data_test.shape)


In [ ]:
# top rows
data_train.head(2)

In [ ]:
data_test.head(2)

In [ ]:
# What are the column names
print("Column Names")
data_train.columns

In [ ]:
data_test.columns

In [ ]:
# what are the data types of the columns?
print("Data Types of Column")
print(data_train.dtypes)


In [ ]:
data_test.dtypes

In [ ]:
data_train.describe() #seems useless

In [ ]:
# checking for null values
data_train.isnull().values.sum()

In [ ]:
data_test.isnull().values.sum()

Ready for Take-Off (Pre-Analysis does not hint at any further action. NULL Values, Character Encoding, etc)

In [ ]:
# split the data set
X_train = np.array(data_train.iloc[:,1:])
y_train = np.array(data_train.iloc[:,0])

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test = np.array(data_test.iloc[:,1:])
y_test = np.array(data_test.iloc[:,0])

In [ ]:
X_test.shape

In [ ]:
# sample rows
X_test[0:2,:]

In [ ]:
y_test.shape

In [ ]:
# sample rows
y_test[0:2]

In [ ]:
from sklearn.svm import SVC

In [ ]:
# takes way too long ...
# commenting out ...

#clf = SVC() 
#clf.fit(X_train,y_train)

In [ ]:
#clf.score(X_train,y_train)

In [ ]:
#y_predict = clf.predict(X_test)
#accuracy_score(y_predict,y_test)

PCA to reduce dimensionality 

[1] states:
One of the things that you can do to speed up the fitting of a machine learning algorithm is changing the optimization algorithm. 
A more common way of speeding up a machine learning algorithm is by using Principal Component Analysis (PCA). If your learning algorithm is too slow because the input dimension is too high, then using PCA to speed it up can be a reasonable choice

[1]: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

[1] PCA is effected by scale so you need to scale the features in your data before applying PCA. Use StandardScaler to help you standardize the dataset’s features onto unit scale (mean = 0 and variance = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardizing the features
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
X_train_scaled[0:2,]

Notice the code below has .95 for the number of components parameter. It means that scikit-learn choose the minimum number of principal components such that 95% of the variance is retained.

Changing it to optional parameter with default value. In the calling code, will reduce it to .5 or 50%

In [ ]:
from sklearn.decomposition import PCA
def get_pca(x_train, x_test,pc_comp=0.95):
    pca = PCA(n_components=pc_comp)
    pca.fit(x_train)
    print("Explained Variance Ratios:", pca.explained_variance_ratio_)
    print("N Components are: ", pca.n_components_)
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    print(x_train.shape, x_test.shape)
    return x_train, x_test

In [ ]:
X_train_sd_pca, X_test_sd_pca = get_pca(X_train_scaled,X_test_scaled,0.5)

50% variance is explained by 39 (transformed) features compared to 784 (original) features

Lets start small and use Log Reg first

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# all parameters not specified are set to their defaults
# default solver is incredibly slow which is why it was changed to 'lbfgs'

clf_logReg = LogisticRegression(solver = 'lbfgs')

In [ ]:
clf_logReg.fit(X_train_sd_pca, y_train)

In [ ]:
# Predict for One Observation (image)
clf_logReg.predict(X_test_sd_pca[0].reshape(1,-1))

In [ ]:
clf_logReg.score(X_test_sd_pca,y_test)

Time to go back to SVC

1. Use GridSearchCV() to tune the parameter 'C' of linear SVM.
1. Then use Non linear SVM ('RBF') and tune 'C' using GridSearchCV.
1. Tune the parameters using training data set and finally test using test data set.
1. You can use PCA for dimension reduction

In [ ]:
# picking these parameters at random to give SVM a spin first
clf_svm = SVC(kernel='linear', C=0.1)
clf_svm

In [ ]:
clf_svm.fit(X_train_sd_pca, y_train)

In [ ]:
y_pred_svm = clf_svm.predict(X_test_sd_pca)

In [ ]:
print("SVM accuracy for TRAIN: ", clf_svm.score(X_train_sd_pca,y_train))
print("SVM accuracy for TEST: ", clf_svm.score(X_test_sd_pca,y_test))

print("LOG-REG accuracy for TRAIN: ", clf_logReg.score(X_train_sd_pca,y_train))
print("LOG-REG accuracy for TEST: ", clf_logReg.score(X_test_sd_pca,y_test))


In [ ]:
# 1. Use GridSearchCV() to tune the parameter 'C' of linear SVM.

# create a parameter grid: map the parameter names to the values that should be searched
# simply a python dictionary
# key: parameter name
# value: list of values that should be searched for that parameter
# single key-value pair for param_grid

Cs = [0.001, 0.01, 0.1, 1, 10,100,500]
paramGrid = {'C':Cs}
#paramGrid = {'C':np.arange(0.01,500,10)}
#print(paramGrid)

# the cv with 5 does not finish, reducing it down to 2
#grid_search = GridSearchCV(clf_svm, paramGrid, cv=5)
grid_search = GridSearchCV(clf_svm, paramGrid, cv=2)

In [ ]:
grid_search

In [81]:
# In previous iteration, even 0.5 PCAed data could not finish execution of Grid Search on Kaggle
# let us reduce it down to .25
# X_train_sd_pca2, X_test_sd_pca2 = get_pca(X_train_scaled,X_test_scaled,0.25)

# let us reduce it down to .15
# X_train_sd_pca2, X_test_sd_pca2 = get_pca(X_train_scaled,X_test_scaled,0.15)

# With Incremental Learning, trying again .25
X_train_sd_pca2, X_test_sd_pca2 = get_pca(X_train_scaled,X_test_scaled,0.25)


# need to combine scaled version of X_test and X_train
# X = np.vstack((X_test_sd_pca2,X_train_sd_pca2))
X = np.concatenate((X_test_sd_pca2,X_train_sd_pca2))

# need to combine original version of y_test and y_train
#y = np.vstack((y_test,y_train))

# diagnostic
print("the shapes of y are:", y_train.shape, y_test.shape)
print("the type of y are:", y_train.dtype, y_test.dtype)
print("the dim of y are:", y_train.ndim, y_test.ndim)

# np.vstack needs the length of two 1D arrays to be same, so switching to concatenate
y = np.concatenate((y_test,y_train))



Explained Variance Ratios: [0.05646717 0.04078272 0.0373938  0.02885115 0.02521109 0.0219427
 0.01923344 0.01745799 0.01535092]
N Components are:  9
(60000, 9) (10000, 9)
the shapes of y are: (60000,) (10000,)
the type of y are: int64 int64
the dim of y are: 1 1


In [ ]:
# Grid Search 

#grid_search.fit(X, y)
#print("the best C value we get is: ",grid_search.best_params_)

In [ ]:
# Grid Search - despite lowering variance retained in PCA and few value of C is too slow

# Alternate is to try "Incremental Learning"
# Details here :https://scikit-learn.org/0.15/modules/scaling_strategies.html

# new classifier - Stochastic Gradient with loss function hinge for SVC
from sklearn.linear_model import SGDClassifier
clf_sgd_svc = SGDClassifier()

clf_sgd_svc


In [82]:
# break train and test set into chunks

# diagnostic
print("the shape of X,y are:", X.shape, y.shape)
print("the type of X,y are:", X.dtype, y.dtype)
print("the dim of X,y are:", X.ndim, y.ndim)

the shape of X,y are: (70000, 9) (70000,)
the type of X,y are: float64 int64
the dim of X,y are: 2 1


In [83]:
# labels of all classes
class_labels = np.unique(y)
print(class_labels)

[0 1 2 3 4 5 6 7 8 9]


In [84]:
chunk_size = 50
X_size = X.shape[0]
iterations = (X_size / chunk_size)
#print(iterations)



for i in range(1,X_size,chunk_size):
    # outer loop 
    for j in range(i,i+chunk_size):
        # inner loop
        # add chunk row to the partial fit
        #print("second-loop")
        clf_sgd_svc.partial_fit(X[i:i+chunk_size,],y[i:i+chunk_size,], class_labels)
        #print(j)





ValueError: Number of features 9 does not match previous data 4.

In [ ]:
print(clf_sgd_svc.score(X,y))

In [ ]:
#help(range)

In [ ]:
# try bagging and boosting
# https://www.kaggle.com/mayankkestwal10/ensemble-learning-bagging-and-boosting